<a href="https://colab.research.google.com/github/saurabhollekat/ey_openai_training/blob/main/Medical_Assistant_Chat_%26_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==1.13.3

In [2]:
from openai import AzureOpenAI
import requests

In [23]:
chat_azure_oai_endpoint = ""
chat_azure_oai_key = ""
chat_azure_oai_deployment = ""
image_azure_oai_deployment = ""

# Only modify, if OpenAI resource is different for Dall-e
image_azure_oai_endpoint = chat_azure_oai_endpoint
image_azure_oai_key = chat_azure_oai_key

api_version = "2024-02-15-preview"

In [ ]:
prompt = input("\nType your query: ")

In [ ]:
chat_client = AzureOpenAI(
        azure_endpoint = chat_azure_oai_endpoint,
        api_key = chat_azure_oai_key,
        api_version = api_version
        )

messages_array = [ {"role": "system", "content": "You are a helpful medical assistant."},
        {"role": "user", "content": "I need some medical advise."},
        {"role": "assistant", "content": "Sure, please help me understand what are you suffering from."},
        {"role": "user", "content": prompt}
        ]

chat_response = chat_client.chat.completions.create(
    model = chat_azure_oai_deployment,
    temperature = 0.7,
    max_tokens = 1200,
    messages = messages_array
)
generated_text = chat_response.choices[0].message.content

messages_array.append({"role": "system", "content": generated_text})

# Print generated text
print("Summary: " + generated_text + "\n")

In [ ]:
# Call the DALL-E model
url = "{}openai/deployments/{}/images/generations?api-version={}".format(image_azure_oai_endpoint, image_azure_oai_deployment, api_version)
headers= { "api-key": image_azure_oai_key, "Content-Type": "application/json" }
body = {
            "prompt": generated_text,
            "n": 1,
            "size": "1024x1024"
        }

image_response = requests.post(url, headers=headers, json=body)
image_revised_prompt = image_response.json()['data'][0]['revised_prompt']
image_url = image_response.json()['data'][0]['url']

 # Display the URL for the generated image
print(image_revised_prompt)
print(image_url)

save_path = 'image_name.jpg'

# Download the image and save it to the specified path
blob_response = requests.get(image_url)
if blob_response.status_code == 200:
    with open(save_path, 'wb') as f:
        f.write(blob_response.content)
    print(f"Image saved to {save_path}")
else:
    print("Failed to download the image")